In [1]:
# %pip install --upgrade pip
# %pip install fairseq2 sonar-space
# %pip install libsndfile

In [2]:
import torch

from Encoders.text.SONAR import SONAR as SONAR_ENC
from Encoders.text.SONAR import SONAR as SONAR_DEC
from Encoders.text.BART import BART as BART_ENC
from Decoders.text.BART import BART as BART_DEC